In [ ]:
#!pip install q keras==2.2.4
#!pip install tensorflow==1.14.0
!pip install tensorflow==2.8.0
#!pip uninstall tensorflow

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from tensorflow import keras

In [ ]:
!nvidia-smi

In [ ]:
!gdown --id 154RWNSZndw3BGTtVNXsOyzQN0LIOPtuA
!gdown --id 1VUh3-dczbssimcD0FwX731eIgvxqSB9g

### Preparar datos

In [ ]:
X_raw = np.load('/content/X.npy')
X_raw = X_raw/255.
y = np.load('/content/y.npy')
X = []
size = (28,28)
for x in X_raw:
    X.append(resize(x, size))
X = np.array(X)

In [ ]:
plt.figure()
plt.title("Cantidad de muestras x dígito")
plt.ylabel("Cantidad")
plt.xlabel("Dígito")
plt.hist(y, bins=10)
plt.plot()

### Visualizando el `dataset`

In [ ]:
plt.figure(figsize=(20,10))
for i in range(25):
    plt.subplot(5,5, i+1)
    idx = np.random.choice(X.shape[0], 1)[0]
    plt.title('id:{} val:{}'.format(idx, y[idx] ))
    plt.imshow(-X[idx], cmap='gray')
    plt.axis('off')
plt.show()

### Split de train-test

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
#cambiar a numeros
y_float = np.where(y == 'pi', 0,
                   np.where(y == 'beta', 1,
                            np.where(y == 'gamma', 2,
                                     np.where(y == 'delta', 3, 4))))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_float, test_size=0.20, random_state=42, stratify=y)

In [ ]:
plt.figure()
plt.title("Cantidad de muestras x dígito (train)")
plt.ylabel("Cantidad")
plt.xlabel("Dígito")
plt.hist(y_train, bins=10)
plt.plot()

In [ ]:
plt.figure()
plt.title("Cantidad de muestras x dígito (test)")
plt.ylabel("Cantidad")
plt.xlabel("Dígito")
plt.hist(y_test, bins=10)
plt.plot()

In [ ]:
# Ajustamos las dimensiones de las imágenes
if X_train.ndim == 3:
    X_train = X_train[...,None]
    X_test = X_test[..., None]
    print(X_train.shape, X_test.shape)

## Definimos un modelo

In [ ]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.optimizers import *

bs = 16
lr = 0.0005


model = Sequential([Conv2D(32, 3, activation='relu', input_shape=(*size,1)),
                    MaxPool2D(),
                    Conv2D(64, 3, activation='relu', padding ='same'),
                    MaxPool2D(),
                    Conv2D(128, 3, activation='relu', padding ='same'),
                    MaxPool2D(),
                    Flatten(),
                    Dense(128, activation='relu'),
                    Dense( 5, activation='softmax')])

#optimizer1='SGD'
model.compile(optimizer= "SGD", loss='sparse_categorical_crossentropy' , metrics=['accuracy'])

model.summary()

## Entrenamiento

In [ ]:
log = model.fit(X_train, y_train, batch_size=bs, epochs=400, validation_data=(X_test, y_test))

In [ ]:
def show_results(model, log):
    loss, acc = model.evaluate(X_test, y_test, batch_size=512, verbose=False)
    print(f'Loss     = {loss:.4f}')
    print(f'Accuracy = {acc:.4f}')
    val_loss = log.history['val_loss']
    val_acc = log.history['val_accuracy'] # val_acc -> val_accuracy
    fig, axes = plt.subplots(1, 2, figsize=(14,4))
    ax1, ax2 = axes
    ax1.plot(log.history['loss'], label='train')
    ax1.plot(val_loss, label='test')
    ax1.set_xlabel('epoch'); ax1.set_ylabel('loss')
    ax2.plot(log.history['accuracy'], label='train')  #acc -> accuracy
    ax2.plot(val_acc, label='test')
    ax2.set_xlabel('epoch'); ax2.set_ylabel('acc')
    for ax in axes: ax.legend()

In [ ]:
show_results(model, log)

## Probando el modelo

In [ ]:
idx = np.random.choice(X_test.shape[0], 1)[0]
im = X_test[idx]
label = y_test[idx]
plt.title('(test) id:{} val:{}'.format(idx, label ))
plt.axis('off')
plt.imshow(-im[:,:,0], cmap='gray')
plt.show()

## Usando el modelo entrenado

In [ ]:
salida = model.predict(im[None,:,:,:])[0]
salida

In [ ]:
plt.figure()
plt.title("Predicción: {} Label: {}".format(salida.argmax(), label ))
plt.ylabel("Probabilidad")
plt.xlabel("Dígito")
plt.ylim([0,1])
plt.bar(np.arange(5), salida)
plt.plot()

In [ ]:
#Descargamos el modelo entrenado.
model.save('modelo_entrenado.h5')
from google.colab import files
files.download('modelo_entrenado.h5')


In [ ]:
from keras.models import load_model

# Carga el modelo desde el archivo .h5
modelo_entrenado = load_model('/content/modelo_entrenado.h5')

# Muestra un resumen del modelo
modelo_entrenado.summary()